In [ ]:
# 1. Install Unsloth & modern dependencies
!pip install unsloth
!pip install --no-deps "trl<0.13.0" peft accelerate bitsandbytes

In [ ]:
# 2. Mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

# Set output directory to Google Drive
output_dir = "/content/drive/MyDrive/mental_health_training"

In [ ]:
# 3. Login to Hugging Face (for pushing to Hub)
from huggingface_hub import login
login()  # This will prompt for your HF token

# Preventing Colab Disconnections

To prevent Google Colab from disconnecting due to inactivity during long training sessions:

1. Press `Ctrl+Shift+I` in your browser to open Developer Tools
2. Go to the Console tab
3. Paste this JavaScript code and press Enter:

```javascript
function ClickConnect(){
    console.log("Keeping session alive...");
    document.querySelector("colab-connect-button").click();
}
setInterval(ClickConnect, 60000);
```

This will click the connect button every 60 seconds to simulate activity.

## Training Strategy for Free Tier

- **Incremental Training**: Train in sessions of ~400 steps each
- **Resume Automatically**: The `resume_from_checkpoint=True` will find the latest checkpoint
- **Backup**: Checkpoints are saved to both Google Drive and Hugging Face Hub
- **Monitor Progress**: Check your Hugging Face model page for training logs

Remember to update `hub_model_id` in the config with your actual Hugging Face username!

downloading the model

In [ ]:
from unsloth import FastLanguageModel
import torch

# Configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True

# FIX: Corrected model name for the DeepSeek Reasoning Distill
model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
)

print("SUCCESS: Model and adapters are loaded.")

In [ ]:
from datasets import load_dataset

# 1. Load the dataset
dataset = load_dataset("ShenLab/MentalChat16K", split="train")

# Blunt check: Print the column names to see what we are working with
print(f"Actual columns in dataset: {dataset.column_names}")

# 2. Define the DeepSeek Reasoning Template
prompt_style = """<｜User｜>{input}<｜Assistant｜><think>
{reasoning}
</think>{output}<｜end of sentence｜>"""

def formatting_prompts_func(examples):
    # MentalChat16K usually uses 'instruction', 'input', and 'output'
    # But sometimes datasets use 'question' or 'context'.
    # This logic picks the right one or defaults to empty string.

    # We will combine instruction and input into one prompt for the model
    instructions = examples.get("instruction", [""] * len(examples["output"]))
    user_inputs  = examples.get("input", [""] * len(examples["output"]))
    outputs      = examples.get("output", [""] * len(examples["output"]))

    texts = []
    for inst, user_in, output in zip(instructions, user_inputs, outputs):
        # Combine instruction and input for a complete prompt
        full_input = f"{inst} {user_in}".strip()

        # Format using the DeepSeek-specific markers
        text = prompt_style.format(
            input=full_input,
            reasoning="The user is seeking mental health support. I should respond with empathy and therapeutic validation.",
            output=output
        )
        texts.append(text)
    return { "text" : texts, }

# 3. Map and Split (95% Training, 5% Validation)
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset = dataset.train_test_split(test_size = 0.05, seed = 3407)

print(f"SUCCESS: Data ready!")
print(f"Training samples: {len(dataset['train'])} | Eval samples: {len(dataset['test'])}")

In [ ]:
training


In [ ]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Configure training with checkpointing for incremental training
sft_config = SFTConfig(
    output_dir=output_dir,
    save_strategy="steps",
    save_steps=100,          # Saves a checkpoint every 100 steps
    save_total_limit=2,      # Keeps only the 2 most recent checkpoints
    max_steps=1800,          # Total training steps (adjust based on your dataset)
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    eval_strategy="steps",
    eval_steps=20,
    push_to_hub=True,        # Upload checkpoints to Hugging Face Hub
    hub_model_id="your-username/mental-health-deepseek-v1",  # Replace with your HF username/model name
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    args=sft_config,
)

# Start training with resume capability
trainer.train(resume_from_checkpoint=True)